### LEEFI Fire pulse analysis

In [1]:
import os
import pandas as pd
import numpy as np
#from scipy.interpolate import interp1d
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt

In [2]:
# read CSV, needed to edit the file to remove extra junk
#csv1 = pd.read_csv('TEST1 800V FIRE.csv', header=None, usecols=[3,4])
#csv1 = pd.read_csv('TEST1 800V FIRE.csv', usecols=[3,4])
csv1 = pd.read_csv('TEST1 800V FIREa.csv')

In [3]:
time = csv1['Time'].values
time = time.astype(float)
voltage = csv1['Voltage'].values
voltage = voltage.astype(float)

In [6]:
print('number of points in the data set: {:,d}'.format(len(voltage)))
print('peak voltage: {:.3f}'.format(voltage.max()))

number of points in the data set: 25,000
peak voltage: 1.060


plot the data to ckeck limits

In [51]:
peak_index = voltage.argmax()
start = peak_index-2500
end = peak_index+5000
x = time[start:end]
y = voltage[start:end]

In [53]:
# plot the fire pulse
plt.plot(x,y,'ro', markersize=1)
plt.xlabel('Time, sec')
plt.ylabel('fire pulse, volts')
plt.show()

### Find the offset voltage.
From the peak time t, find the averge from t-300 to t-200.

In [54]:
peak_index = voltage.argmax()
start = peak_index-3000
end = peak_index-2500
x = time[start:end]
y = voltage[start:end]
offset= y.mean()
print('voltage offset: {:.3f} volts'.format(offset))

voltage offset: -0.011 volts


In [55]:
# plot base line data, pre pulse, with offset
plt.plot(x,y,'ro', markersize=4)
plt.xlabel('Time, nsec')
plt.ylabel('Voltage')
plt.legend(['data'], loc='best')
plt.show()

In [56]:
start = peak_index-2500
end = peak_index+5000
x = time[start:end]
y = voltage[start:end]
y -= offset  #remove the offset

A [Savitzky–Golay](https://en.wikipedia.org/wiki/Savitzky%E2%80%93Golay_filter) filter is a digital filter that can be applied to a set of digital data points for the purpose of smoothing the data, that is, to increase the signal-to-noise ratio without greatly distorting the signal. This is achieved, in a process known as convolution, by fitting successive sub-sets of adjacent data points with a low-degree polynomial by the method of linear least squares. When the data points are equally spaced, an analytical solution to the least-squares equations can be found, in the form of a single set of "convolution coefficients" that can be applied to all data sub-sets, to give estimates of the smoothed signal, (or derivatives of the smoothed signal) at the central point of each sub-set. The method, based on established mathematical procedures, was popularized by Abraham Savitzky and Marcel J. E. Golay who published tables of convolution coefficients for various polynomials and sub-set sizes in 1964.

In [57]:
ynew = savgol_filter(y, 51, 3)  # filter the data

In [59]:
# plot the fire pulse
plt.plot(x,y,'ro', markersize=1)
#plt.plot(x,ynew)
plt.xlabel('Time, nsec')
plt.ylabel('fire pulse, volts')
plt.legend(['data'], loc='best')
plt.show()

### find the following:
1. peak voltage
2. average pre pulse voltage level
3. index of 90% of peak
4. find 10% index
5. calculate rise time
6. calculate peak current

In [60]:
peak = y.max()
peak_index = y.argmax()+10
print('peak voltage: {:.3f}'.format(peak))
print('index of peak: {:.3f}'.format(peak_index))

peak voltage: 1.083
index of peak: 2510.000


#### limit data set to index of peak

In [62]:
x1 = x[50:peak_index]-x[50]
y1 = y[50:peak_index]

# plot the fire pulse
y1_fit = savgol_filter(y1, 51, 3)
#plt.plot(x1,y1,'ro', markersize=1)
plt.plot(x1,y1_fit)
plt.xlabel('Time, nsec')
plt.ylabel('fire pulse, volts')
plt.legend(['Savitzky–Golay curve fit'], loc='best')
plt.show()

#### find rise time

In [63]:
index_90pc = (np.abs(y1_fit-y1_fit.max()*0.9)).argmin()
index_10pc = (np.abs(y1_fit-y1_fit.max()*0.1)).argmin()
rise_time = x1[index_90pc]-x1[index_10pc]

print('peak voltage value: {:.3f} volts'.format(y1_fit.max()))
print('90% voltage value: {:.3f} volts at {:.3f} ns'.format(y1_fit[index_90pc], x1[index_90pc]*1e9))
print('10% voltage value: {:.3f} volts at {:.3f} ns'.format(y1_fit[index_10pc], x1[index_10pc]*1e9))
print('rise time: {:.3f} ns'.format(rise_time*1e9))

peak voltage value: 1.077 volts
90% voltage value: 0.969 volts at 178.400 ns
10% voltage value: 0.108 volts at 103.040 ns
rise time: 75.360 ns


#### plot the 90% and 10% data points

In [64]:
# plot the fire pulse
#plt.plot(x1,y1,'ro', markersize=1)
plt.plot(x1,y1_fit)
plt.plot(x1[index_90pc],y1_fit[index_90pc],'x',markersize=15)  # 90% point
plt.plot(x1[index_10pc],y1_fit[index_10pc],'x',markersize=15)  # 10% point
plt.xlabel('Time, nsec')
plt.ylabel('fire pulse, volts')
plt.legend(['Savitzky–Golay curve fit','90%','10%'], loc='best')
plt.show()

In [67]:
R1 = 5.462
R2 = 0.005065
print('peak current: {:,.0f} amps'.format(peak/(R2/R1)))
print('capacitor voltage: {:,.0f} volts'.format(800))

peak current: 1,168 amps
capacitor voltage: 800 volts
